In [80]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [81]:
data_dir = ""

# Path to data
patients_path =  data_dir +'patients.csv'
admissions_path = data_dir +'admissions.csv'
diagnoses_path = data_dir +'diagnoses_icd.csv'
lab_events_path = data_dir +'labevents_sample.csv'
d_icd_labs_path =data_dir +'d_labitems.csv'
d_icd_diagnoses_path = data_dir +'d_icd_diagnoses.csv'
d_icd_procedures_path = data_dir +'d_icd_procedures.csv'
procedures_path =data_dir + 'procedures_icd.csv'
prescriptions_path = data_dir +'prescriptions_sample.csv'
notes_path =data_dir + 'Notes.csv'

# Load the data
patients = pd.read_csv(patients_path, usecols=['subject_id', 'gender'])
admissions = pd.read_csv(admissions_path, usecols=['subject_id', 'hadm_id', 'race'])
diagnoses = pd.read_csv(diagnoses_path, usecols=['subject_id', 'hadm_id', 'icd_code'])
d_icd_diagnoses = pd.read_csv(d_icd_diagnoses_path, usecols=['icd_code', 'long_title'])
lab_events = pd.read_csv(lab_events_path, usecols=['subject_id', 'hadm_id', 'itemid', 'valuenum', 'ref_range_lower','ref_range_upper','flag'])
d_icd_labs = pd.read_csv(d_icd_labs_path, usecols=['itemid', 'label'])
procedures = pd.read_csv(procedures_path, usecols=['subject_id', 'hadm_id', 'icd_code'])
d_icd_procedures = pd.read_csv(d_icd_procedures_path, usecols=['icd_code', 'long_title'])
prescriptions = pd.read_csv(prescriptions_path, usecols=['subject_id', 'hadm_id', 'drug', 'dose_val_rx','dose_unit_rx'],  encoding='utf-16', on_bad_lines='skip')
notes = pd.read_csv(notes_path, usecols=['subject_id', 'hadm_id', 'Symptoms', 'allergies'])

In [82]:
threshold = 0.0001
min_count = max(1, int(threshold * len(admissions['hadm_id'].unique())))
print(f"Minimum count for frequent itemsets: {min_count}")

Minimum count for frequent itemsets: 54


In [83]:
# Filter Diagnoses
# Count frequency of each diagnosis code
diag_counts = diagnoses['icd_code'].value_counts()
# Get only frequent codes
frequent_diags = diag_counts[diag_counts >= min_count].index
# Filter diagnoses
diagnoses = diagnoses[diagnoses['icd_code'].isin(frequent_diags)]
print(f"Remaining diagnoses: {diagnoses['icd_code'].nunique()}")

#import frequencies to csv
frequencies = pd.DataFrame(diagnoses['icd_code'].value_counts())
frequencies.to_csv('diagnoses_frequencies.csv', index=True)

Remaining diagnoses: 6332


In [84]:
# Count frequency of each procedure code
proc_counts = procedures['icd_code'].value_counts()

# Get only frequent codes
frequent_procs = proc_counts[proc_counts >= min_count].index

# Filter procedures
procedures = procedures[procedures['icd_code'].isin(frequent_procs)]
print(f"Remaining procedures: {procedures['icd_code'].nunique()}")

#import frequencies to csv
# frequencies = pd.DataFrame(procedures['icd_code'].value_counts())
# frequencies.to_csv('procedures_frequencies.csv', index=True)


Remaining procedures: 1701


In [85]:
# Count number of labevents item ids with abnormal flag

lab_counts = lab_events['itemid'].value_counts()

# Get frequent lab result patterns
frequent_labs = lab_counts[lab_counts >= min_count].index

# Filter lab events
lab_events = lab_events[
    lab_events['itemid'].isin(frequent_labs)
]
print(f"Remaining lab results: {lab_events['itemid'].nunique()}")

#import frequencies to csv
frequencies = pd.DataFrame(lab_events['itemid'].value_counts())
frequencies.to_csv('lab_events_frequencies.csv', index=True)


Remaining lab results: 254


In [86]:
pres_counts = prescriptions['drug'].value_counts()
print(f"Total prescriptions: {len(pres_counts)}")
# Get only frequent codes
frequent_pres = pres_counts[pres_counts >= min_count].index
# Filter prescriptions
prescriptions = prescriptions[prescriptions['drug'].isin(frequent_pres)]
print(f"Remaining prescriptions: {prescriptions['drug'].nunique()}")

Total prescriptions: 561
Remaining prescriptions: 43


In [87]:
print("Merging dataframes...")

# Merge diagnoses with d_icd_diagnoses
diagnoses = diagnoses.merge(d_icd_diagnoses[['icd_code', 'long_title']], 
                            on='icd_code', how='left')
print(f"Diagnoses after merge: {diagnoses.shape}")
print(diagnoses.head())

# Merge procedures with d_icd_procedures
procedures = procedures.merge(d_icd_procedures[['icd_code', 'long_title']], 
                              on='icd_code', how='left')
print(f"Procedures after merge: {procedures.shape}")
print(procedures.head())

# Merge lab_events with d_icd_labs (d_labitems)
lab_events_with_desc = lab_events.merge(d_icd_labs[['itemid', 'label']], 
                                       on='itemid', how='left')
print(f"Lab events after merge: {lab_events_with_desc.shape}")

# Process lab events (abnormal results only, with range status)
print("Processing lab events...")
lab_events_with_desc['lab_result'] = (
    lab_events_with_desc['itemid'].astype(str) + '_' + 
    lab_events_with_desc['label'].fillna('Unknown')
)
lab_events_with_desc = lab_events_with_desc.dropna(subset=['hadm_id', 'flag'])
print(f"Filtered lab events (non-null flag): {lab_events_with_desc.shape}")

if lab_events_with_desc.empty:
    print("WARNING: No abnormal lab events found.")
    lab_grouped = pd.DataFrame(columns=['hadm_id', 'labs'])
else:
    # Categorize lab results as Below, Above, or Unknown
    def classify_range(row):
        if pd.notnull(row['valuenum']) and pd.notnull(row['ref_range_lower']) and row['valuenum'] < row['ref_range_lower']:
            return 'Below'
        elif pd.notnull(row['valuenum']) and pd.notnull(row['ref_range_upper']) and row['valuenum'] > row['ref_range_upper']:
            return 'Above'
        return 'Unknown'

    lab_events_with_desc['range_status'] = lab_events_with_desc.apply(classify_range, axis=1)
    lab_events_with_desc['lab_result'] = (
        lab_events_with_desc['itemid'].astype(str) + '_' + 
        # lab_events_with_desc['label'].fillna('Unknown') + '_' + 
        lab_events_with_desc['range_status']
    )
    lab_events_with_desc = lab_events_with_desc[lab_events_with_desc['range_status'] != 'Unknown']
    print(f"Lab events after filtering Unknown status: {lab_events_with_desc.shape}")


# Group lab events by hadm_id
lab_grouped = (lab_events_with_desc.groupby('hadm_id')['lab_result']
               .apply(lambda x: list(x.unique()))
               .reset_index()
               .rename(columns={'lab_result': 'labs'}))
print(f"Lab grouped shape: {lab_grouped.shape}")
print(lab_grouped.head())


Merging dataframes...
Diagnoses after merge: (6278083, 4)
   subject_id   hadm_id icd_code  \
0    10000032  22595853     5723   
1    10000032  22595853    78959   
2    10000032  22595853     5715   
3    10000032  22595853    07070   
4    10000032  22595853      496   

                                          long_title  
0                                Portal hypertension  
1                                      Other ascites  
2      Cirrhosis of liver without mention of alcohol  
3  Unspecified viral hepatitis C without hepatic ...  
4  Chronic airway obstruction, not elsewhere clas...  
Procedures after merge: (764313, 4)
   subject_id   hadm_id icd_code  \
0    10000032  22595853     5491   
1    10000032  22841357     5491   
2    10000032  25742920     5491   
3    10000068  25022803     8938   
4    10000117  27988844  0QS734Z   

                                          long_title  
0                    Percutaneous abdominal drainage  
1                    Percutaneou

In [88]:
procedures['combined_title'] = (
    procedures['icd_code'].astype(str)
    # procedures['long_title'].fillna('Unknown')
)

# Group procedures by hadm_id, collecting unique combined titles
procedures_grouped = (procedures.groupby('hadm_id')['combined_title']
                      .apply(lambda x: list(x.unique()))
                      .reset_index()
                      .rename(columns={'combined_title': 'procedures'}))

# Print shape and sample of grouped procedures
print(f"Procedures grouped shape: {procedures_grouped.shape}")
print(procedures_grouped.head())

Procedures grouped shape: (274352, 2)
    hadm_id                                     procedures
0  20000041                                         [8154]
1  20000045                                      [3E0436Z]
2  20000069                             [0KQM0ZZ, 10E0XZZ]
3  20000102                                   [7359, 7309]
4  20000147  [02100Z9, B211YZZ, 021209W, 06BQ4ZZ, 5A1221Z]


In [89]:
diagnoses['combined_title'] = (
    diagnoses['icd_code'].astype(str)
    # diagnoses['long_title'].fillna('Unknown')
)

# Group diagnoses by hadm_id
diagnoses_grouped = (diagnoses.groupby('hadm_id')['combined_title'] #icd_code
                     .apply(lambda x: list(x.unique()))
                     .reset_index()
                     .rename(columns={'combined_title': 'diagnoses'}))
print(f"Diagnoses grouped shape: {diagnoses_grouped.shape}")
print(diagnoses_grouped.head())

Diagnoses grouped shape: (544361, 2)
    hadm_id                                          diagnoses
0  20000019  [0389, 59080, 75310, 5849, 2761, 99591, 25000,...
1  20000024  [D500, K521, I10, E538, M810, R270, Z9181, H54...
2  20000034  [K831, K8689, K861, K869, R1032, R8279, J439, ...
3  20000041  [71536, 25002, V8541, 4019, 2724, V4586, 53081...
4  20000045  [A419, N390, C7951, C787, K56699, C779, K5100,...


In [90]:
prescriptions['combined_title'] = (
    prescriptions['drug'].astype(str) + '_' + 
    prescriptions['dose_val_rx'].fillna('Unknown').astype(str) + '_' + 
    prescriptions['dose_unit_rx'].fillna('Unknown').astype(str)
)


# Group prescriptions by hadm_id
prescriptions_grouped = (prescriptions.groupby('hadm_id')['combined_title']
                          .apply(lambda x: list(x.unique()))
                          .reset_index()
                          .rename(columns={'combined_title': 'prescriptions'}))
print(f"Prescriptions grouped shape: {prescriptions_grouped.shape}")

# import as csv file
# prescriptions_grouped.to_csv('prescriptions_grouped.csv', index=False)

print(prescriptions_grouped.head())



Prescriptions grouped shape: (252, 2)
    hadm_id                                      prescriptions
0  20030125  [Omeprazole_20_mg, OxycoDONE (Immediate Releas...
1  20032235  [Morphine Sulfate_2-4_mg, Lorazepam_0.5_mg, He...
2  20144849  [Sodium Chloride 0.9%  Flush_3-10_mL, Acetamin...
3  20195471  [Warfarin_7.5_mg, Omeprazole_20_mg, HYDROmorph...
4  20214994  [HYDROmorphone (Dilaudid)_0.25_mg, Potassium C...


In [91]:
notes = pd.read_csv(notes_path, usecols=['hadm_id', 'Symptoms', 'allergies'])

# Preprocess Symptoms and allergies to ensure they are lists with prefixed, formatted items
def format_items(value, prefix):
    if pd.isna(value) or value is None or value == '':
        return []
    try:
        if isinstance(value, str):
            # Split by comma, clean, and format each item
            items = [item.strip().lower().replace(' ', '_') for item in value.split(',') if item.strip() and item.lower() != 'none']
            return [f"{prefix}{item}" for item in items]
        if isinstance(value, list):
            # Clean and format list items
            items = [item.strip().lower().replace(' ', '_') for item in value if isinstance(item, str) and item.strip() and item.lower() != 'none']
            return [f"{prefix}{item}" for item in items]
        print(f"Unexpected value type for {prefix}: {value} (type: {type(value)})")
        return []
    except Exception as e:
        print(f"Error processing {prefix} value {value}: {e}")
        return []

# Apply formatting and ensure no NaN values remain
notes['Symptoms'] = notes['Symptoms'].apply(lambda x: format_items(x, 'Symptom_'))
notes['allergies'] = notes['allergies'].apply(lambda x: format_items(x, 'Allergy_'))

# Group by hadm_id, keeping Symptoms and allergies separate
notes_grouped = notes.groupby('hadm_id').agg({
    'Symptoms': lambda x: list(set(item for sublist in x for item in sublist if isinstance(sublist, list))),
    'allergies': lambda x: list(set(item for sublist in x for item in sublist if isinstance(sublist, list)))
}).reset_index()

# Print shape and sample of grouped notes
print(f"\nNotes grouped shape: {notes_grouped.shape}")
print("Sample of grouped notes:")
print(notes_grouped.head())


Notes grouped shape: (331793, 3)
Sample of grouped notes:
    hadm_id                                           Symptoms  \
0  20000019  [Symptom_fever, Symptom_flank_pain, Symptom_na...   
1  20000024               [Symptom_weakness, Symptom_diarrhea]   
2  20000034                                                 []   
3  20000041                              [Symptom_l_knee_pain]   
4  20000057  [Symptom_cough, Symptom_ankle_pain_(s/p_mechan...   

                                           allergies  
0  [Allergy_no_known_allergies_/_adverse_drug_rea...  
1                                  [Allergy_aspirin]  
2  [Allergy_no_known_allergies_/_adverse_drug_rea...  
3                                    [Allergy_latex]  
4  [Allergy_no_known_allergies_/_adverse_drug_rea...  


In [92]:
# Admissions with patients
admissions_patients = admissions.merge(patients[['subject_id', 'gender']], on='subject_id', how='left')
print(f"Admissions with patients shape: {admissions_patients.shape}")

Admissions with patients shape: (546028, 4)


In [93]:
# Create base dataframe with all hadm_ids
transactions_df = pd.DataFrame({'hadm_id': admissions['hadm_id'].unique()})

# Merge all grouped data
print("Combining data for transactions...")
#transactions_df = transactions_df.merge(admissions_patients[['hadm_id', 'gender', 'race']], on='hadm_id', how='left')
transactions_df = transactions_df.merge(diagnoses_grouped, on='hadm_id', how='left')
transactions_df = transactions_df.merge(procedures_grouped, on='hadm_id', how='left')
# transactions_df = transactions_df.merge(lab_grouped, on='hadm_id', how='left')
transactions_df = transactions_df.merge(prescriptions_grouped, on='hadm_id', how='left')
# transactions_df = transactions_df.merge(notes_grouped, on='hadm_id', how='left')
print(f"Transactions dataframe shape: {transactions_df.shape}")


Combining data for transactions...
Transactions dataframe shape: (546028, 4)


In [94]:
# Generate transactions
transactions = []
for _, row in transactions_df.iterrows():
    transaction = []
    
    # Helper function to safely add items
    def add_items(items, prefix=''):
        if isinstance(items, list):
            transaction.extend([f"{prefix}{item}" for item in items if pd.notna(item) and str(item).strip()])
    
    # Diagnoses
    if 'diagnoses' in row:
        add_items(row['diagnoses'], 'DIA_')
    
    # Procedures
    if 'procedures' in row:
        add_items(row['procedures'], 'PRO_')
    
    # Lab results
    if 'labs' in row:
        add_items(row['labs'], 'LAB_')
    
    # Gender
    if 'gender' in row and pd.notna(row['gender']):
        transaction.append(f"Gender_{row['gender']}")
    
    # Race
    if 'race' in row and pd.notna(row['race']):
        transaction.append(f"Race_{row['race'].replace(' ', '_')}")
    
    # Symptoms
    if 'Symptoms' in row:
        add_items(row['Symptoms'])
    
    # Allergies
    if 'allergies' in row:
        add_items(row['allergies'])

    # Prescriptions
    if 'prescriptions' in row:
        add_items(row['prescriptions'], 'PRE_')
    
    transactions.append(transaction)

# Print sample transactions
print("\nSample transactions:")
for i, t in enumerate(transactions[:10], 1):
    print(f"Transaction {i}: {t[:10]}...")


Sample transactions:
Transaction 1: ['DIA_5723', 'DIA_78959', 'DIA_5715', 'DIA_07070', 'DIA_496', 'DIA_29680', 'DIA_30981', 'DIA_V1582', 'PRO_5491', 'PRE_Furosemide_40_mg']...
Transaction 2: ['DIA_07071', 'DIA_78959', 'DIA_2875', 'DIA_2761', 'DIA_496', 'DIA_5715', 'DIA_V08', 'DIA_3051', 'PRO_5491', 'PRE_Furosemide_40_mg']...
Transaction 3: ['DIA_07054', 'DIA_78959', 'DIA_V462', 'DIA_5715', 'DIA_2767', 'DIA_2761', 'DIA_496', 'DIA_V08', 'DIA_3051', 'DIA_78791']...
Transaction 4: ['DIA_45829', 'DIA_07044', 'DIA_7994', 'DIA_2761', 'DIA_78959', 'DIA_2767', 'DIA_3051', 'DIA_V08', 'DIA_V4986', 'DIA_V462']...
Transaction 5: ['DIA_30500', 'PRO_8938']...
Transaction 6: ['DIA_G3183', 'DIA_F0280', 'DIA_R441', 'DIA_R296', 'DIA_E785', 'DIA_Z8546', 'PRE_Senna_8.6_mg', 'PRE_Sodium Chloride 0.9%  Flush_3-10_mL', 'PRE_Heparin_5000_UNIT', 'PRE_Polyethylene Glycol_17_g']...
Transaction 7: ['DIA_R4182', 'DIA_G20', 'DIA_F0280', 'DIA_R609', 'DIA_E785', 'DIA_Z8546']...
Transaction 8: ['DIA_5283', 'DIA_52109'

In [95]:
# Sort transactions by number of items (descending or ascending as needed)
sorted_transactions = sorted(transactions, key=len, reverse=True)  # Use reverse=False for ascending

# Convert to DataFrame for CSV export
df_sorted_transactions = pd.DataFrame({
    'transaction_id': range(1, len(sorted_transactions) + 1),
    'num_items': [len(t) for t in sorted_transactions],
    'items': [', '.join(t) for t in sorted_transactions]
})

# Save to CSV
# df_sorted_transactions.to_csv('sorted_transactions.csv', index=False)

# Print a few to confirm
print(df_sorted_transactions.head())


   transaction_id  num_items  \
0               1         98   
1               2         96   
2               3         91   
3               4         84   
4               5         82   

                                               items  
0  DIA_T8131XA, DIA_R6521, DIA_J9601, DIA_N179, D...  
1  DIA_8602, DIA_51881, DIA_42843, DIA_5070, DIA_...  
2  DIA_2866, DIA_51881, DIA_5845, DIA_99594, DIA_...  
3  DIA_J441, DIA_K7200, DIA_R579, DIA_J9602, DIA_...  
4  DIA_0383, DIA_78552, DIA_5184, DIA_5845, DIA_5...  


In [96]:
# #Filter Rare Items First
# from collections import Counter

# # Count item frequencies across all transactions
# item_counts = Counter(item for transaction in transactions for item in transaction)

# # Keep only items that appear at least min_freq times
# min_freq = 50  # Adjust based on your data size
# frequent_items = {item for item, count in item_counts.items() if count >= min_freq}

# # Filter transactions
# filtered_transactions = [
#     [item for item in txn if item in frequent_items] 
#     for txn in transactions
# ]

In [97]:
# Create a mask for rows with non-empty transactions
mask = [len(t) > 0 for t in transactions]

# Filter transactions and transactions_df together
filtered_transactions = [t for t in transactions if len(t) > 0]
filtered_df = transactions_df.loc[mask].reset_index(drop=True)

# Now lengths should match
print(len(filtered_transactions), filtered_df.shape[0])

# Create the DataFrame
transactions_df_out = pd.DataFrame({
    'hadm_id': filtered_df['hadm_id'],
    'transaction': filtered_transactions
})

# Save to CSV
transactions_df_out.to_csv("transactions.csv", index=False)
print("Transactions saved to transactions.csv")

544857 544857
Transactions saved to transactions.csv


In [98]:
# Extract the list of transactions from the DataFrame
transactions_list = transactions_df_out['transaction'].tolist()

# Initialize encoder and fit_transform the transactions
encoder = TransactionEncoder()
onehot = encoder.fit_transform(transactions_list)

# Convert to DataFrame for easier analysis
df_onehot = pd.DataFrame(onehot, columns=encoder.columns_)

print(f"One-hot encoded shape: {df_onehot.shape}")

One-hot encoded shape: (544857, 8320)


In [99]:
# Get item frequencies from your one-hot encoded data
item_frequencies = df_onehot.sum().sort_values(ascending=False)

# Display top N frequent items
print("Top Most Frequent Items:")
print(item_frequencies.head(30))

# Convert to DataFrame and reset index to get items as a column
freq_df = item_frequencies.reset_index()
freq_df.columns = ['Item', 'Frequency']  # Name the columns

# Save to CSV with both item names and frequencies
freq_df.to_csv("frequent_items.csv", index=False)

Top Most Frequent Items:
DIA_4019      102362
DIA_E785       84568
DIA_I10        83773
DIA_2724       67288
DIA_Z87891     62803
DIA_K219       56155
DIA_53081      48624
DIA_25000      43076
DIA_F329       41876
DIA_I2510      41548
DIA_F419       38910
DIA_42731      37063
DIA_4280       36606
DIA_311        36349
DIA_41401      36077
DIA_N179       35884
DIA_Z20822     33113
DIA_V1582      31704
DIA_Z7901      30956
DIA_5849       29135
DIA_2449       28519
DIA_E039       27999
DIA_Z794       27640
DIA_E119       26266
DIA_3051       25878
DIA_2859       24392
DIA_F17210     24106
DIA_G4733      23933
DIA_40390      23831
DIA_V5861      22634
dtype: int64


In [100]:
# Removing redundant columns
# Define patterns to exclude
# unwanted_patterns = [
#     'no_known_allergies',
#     'patient_recorded_as_having_no_known_allergies',
#     'Allergy_*********per_pt_has_lots_of_allergies._daughter_will_bring_\nlist***********_/_ampicillin_/_cortisone_/_nitrofurantoin_/',
#     'Allergy____',
#     '_____'  # For the "____" case
# ]

# # Filter transactions to remove these items
# filtered_transactions = [
#     [item for item in txn 
#      if not any(pattern in str(item).lower() for pattern in unwanted_patterns)]
#     for txn in transactions
# ]

In [101]:
#sample 1000 rows from the onehot dataframe
#df_onehot_sample = df_onehot.sample(n=10000, random_state=1)

In [ ]:

print(df_onehot.shape)
#frequent_itemsets = fpgrowth(df_onehot_sample, min_support=0.1, use_colnames=True)
frequent_itemsets = fpgrowth(df_onehot, min_support=0.001, use_colnames=True)
print(f"Frequent itemsets shape: {frequent_itemsets.shape}")
print("Frequent Itemsets:\n", frequent_itemsets)
frequent_itemsets.to_csv("frequent_itemsets.csv", index=False)

(544857, 8320)


In [ ]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
print("\nAssociation Rules:\n", rules)
rules.to_csv("association_rules.csv", index=False)

KeyboardInterrupt: 